In [ ]:
import time
import torch
import torch_xla
import torch_xla.experimental.fori_loop
from torch._higher_order_ops.while_loop import while_loop
import torch_xla.core.xla_model as xm
import torch_xla.core.xla_builder as xb

device = xm.xla_device()

def cond_fn(init, limit_value):
    return limit_value[0] >= init[0]

def body_fn(init, limit_value):
    one_value = torch.ones(1, dtype=torch.float32, device=device)
    return (torch.add(init, one_value), limit_value.clone())

start_time = time.time()

init = torch.tensor([0], dtype=torch.float32, device=device, requires_grad=True)
limit_value = torch.tensor([100], dtype=torch.float32, device=device, requires_grad=True)
res = while_loop(cond_fn, body_fn, (init, limit_value))
print("res: ", res)

# Implement a multiplier using while_loop
# value = 1
# for i in range(10):
#   value = value * 2
# print(value)  # 1024

print("--- %s seconds ---" % (time.time() - start_time))

res:  (tensor([101.], device='xla:0', grad_fn=<Error>), tensor([100.], device='xla:0', grad_fn=<Error>))
--- 0.3206775188446045 seconds ---


In [2]:
assert res
a, b = res
a.backward()
b.backward()

RuntimeError: Autograd not implemented for while_loop

## `fori_loop` also doesn't have gradients (output is detached)

In [3]:
import time
import torch
import torch_xla
import torch_xla.core.xla_model as xm
import torch_xla.core.xla_builder as xb

from torch_xla.experimental.fori_loop import fori_loop

device = xm.xla_device()

lower = torch.tensor(0, device=device)
upper = torch.tensor(50, device=device)
init_val = torch.tensor(1, dtype=torch.float32, device=device, requires_grad=True)

def body_fun(x):
  return torch.add(x, 1.0)

_, res_with_loop = fori_loop(lower, upper, body_fun, (init_val))
res_with_loop


tensor(51., device='xla:0')